<a href="https://colab.research.google.com/github/savancs/product_recommender/blob/tori_branch/Newest_NER_Basic_DistilBERT_DLAIEF23_PR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup - Check GPU!

## Packages/Libraries

In [ ]:
!pip install transformers
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.0/254.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import wandb

from torch.utils.data import DataLoader, TensorDataset

import re

import nltk

In [ ]:
!wandb login --relogin 59add063e6bbf968685833105867f8be02a2191a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Data

In [ ]:
# product data

sephora_data = 'https://raw.githubusercontent.com/torrileigh/sephora_cleaned_11.26.23/main/cosmetic_p11.26.23.csv'
sephora_df = pd.read_csv(sephora_data)

sephora_df.head()

,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,ALGENIST,GENIUS Ultimate Anti-Aging Cream,$112.00,4.3,"Water, Caprylic/Capric Triglyceride, Hydrogent...",1,1,1,1,1
1,Moisturizer,ALGENIST,Overnight Restorative Cream,$94.00,4.4,"Water, Isopropyl Isostearate, Butyrospermum Pa...",0,0,0,0,0
2,Moisturizer,ALGENIST,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,$28.00,4.4,"Cyclopentasiloxane, Water, Alcohol Denat., Gly...",0,0,0,0,0
3,Moisturizer,ALGENIST,POWER Recharging Night Pressed Serum,$95.00,4.4,"Cocos Nucifera (Coconut) Water, Water (Aqua, E...",1,1,1,1,1
4,Moisturizer,ALGENIST,Firming & Lifting Neck Cream,$98.00,3.9,"Water, Isononyl Isononanoate, Dimethicone, But...",0,0,0,0,0


In [ ]:
# ingredient mapping data for skin concerns

ingredient_mapping_data = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/PR%20Text%20Input%20Dataset%20-%20Ingredient%20Mapping.csv'
ingredient_mapping_df = pd.read_csv(ingredient_mapping_data)

ingredient_mapping_df.head()

,Skin Concern,Preferred Ingredients,Excluded Ingredients
0,Sensitive,"Aloe Vera, Argan Oil, Borage Oil, Calendula, C...","Synthetic Fragrance, Essential Oils, Methyliso..."
1,Acne,"Salicylic Acid, Benzoyl Peroxide, Hyaluronic A...","Cocoa Butter, Lanolin, Mineral Oil, Paraffin, ..."
2,Oily,"Salicylic Acid, Niacinamide, Vitamin B3, Honey...","Coconut Oil, Cocoa Butter, Mineral Oil, Silico..."
3,Dry,"Hyaluronic acid, Glycerin, Ceramides, Shea But...","Alcohol, Fragrance, Salicylic Acid, Benzoyl Pe..."
4,Combination,"Hyaluronic acid, Ceramides, Lactic Acid, Antio...","Alcohol, Fragrance, Essential Oils, Benzoyl Pe..."


In [ ]:
#we aren't using this now, may need later.  Keeping just in case

test_val_data = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/GPT%20Dataset%20PR.csv'
test_val_df = pd.read_csv(test_val_data)

test_val_df.head()

,Input Text,Product Type,Min Price,Max Price,Skin Concerns,Preferred Ingredients,Excluded Ingredients
0,Looking for a sun protect for dry skin $36-$70...,Sun Protect,36.0,70.0,Dry,Peptides,NaN
1,Looking for a eye cream for combination skin $...,Eye Cream,18.0,40.0,Combination,NaN,Phthalates
2,Looking for a moisturizer for dry skin $40-$72...,Moisturizer,40.0,72.0,Dry,NaN,Formaldehyde
3,Looking for a treatment for dry skin $20-$40 w...,Treatment,20.0,40.0,Dry,NaN,Parabens
4,Looking for a face mask for acne-prone skin $3...,Face Mask,37.0,44.0,Acne,Aloe Vera,NaN


In [ ]:
# our complete annotations dataset

import json
import requests

url = 'https://raw.githubusercontent.com/savancs/product_recommender/tori_branch/annotations-test%20combo.json'

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    annotated_df = json.loads(response.content)
else:
    print("Failed to download the data")


In [ ]:
print(annotated_df.keys())

dict_keys(['classes', 'annotations'])


In [ ]:
ner_annotations = annotated_df['annotations']
ner_classes = annotated_df['classes']

# checking it loaded well
for i, entry in enumerate(ner_annotations[:10]):
    if entry is not None:
        text, annotation_details = entry
        print(f"Entry {i+1}:")
        print("Text:", text)
        print("Annotations:", annotation_details)
        print("\n")
    else:
        print(f"Entry {i+1}: None\n")


Entry 1: None

Entry 2:
Text: Combination skin eye cream more than $6 but less than $16
Annotations: {'entities': [[0, 11, 'B-SKIN_CONCERN'], [12, 16, 'O'], [17, 20, 'B-PRODUCT_TYPE'], [21, 26, 'I-PRODUCT_TYPE'], [27, 31, 'O'], [32, 36, 'O'], [37, 39, 'B-MIN_PRICE'], [40, 43, 'O'], [44, 48, 'O'], [49, 53, 'O'], [54, 57, 'B-MAX_PRICE']]}


Entry 3:
Text: Cleanser with seaweed more than $41 but less than $95
Annotations: {'entities': [[0, 8, 'B-PRODUCT_TYPE'], [9, 13, 'O'], [14, 21, 'B-PREF_INGREDIENT'], [22, 26, 'O'], [27, 31, 'O'], [32, 35, 'B-MIN_PRICE'], [36, 39, 'O'], [40, 44, 'O'], [45, 49, 'O'], [50, 53, 'B-MAX_PRICE']]}


Entry 4:
Text: What's a good eye cream to use for skin that's sensitive?
Annotations: {'entities': [[0, 4, 'O'], [4, 6, 'O'], [7, 8, 'O'], [9, 13, 'O'], [14, 17, 'B-PRODUCT_TYPE'], [18, 23, 'I-PRODUCT_TYPE'], [24, 26, 'O'], [27, 30, 'O'], [31, 34, 'O'], [35, 39, 'O'], [40, 46, 'O'], [47, 56, 'B-SKIN_CONCERN'], [56, 57, 'O']]}


Entry 5:
Text: I need a face mask 

In [ ]:
ner_annotations = annotated_df['annotations']
ner_classes = annotated_df['classes']

start_index = -10 if len(ner_annotations) >= 10 else -len(ner_annotations)

# Print the last 10 entries
for i, entry in enumerate(ner_annotations[start_index:], start=start_index + len(ner_annotations) + 1):
    if entry is not None:
        text, annotation_details = entry
        print(f"Entry {i}:")
        print("Text:", text)
        print("Annotations:", annotation_details)
        print("\n")
    else:
        print(f"Entry {i}: None\n")

Entry 292:
Text: Looking for a face mask for oily skin $35-$48
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 18, 'B-PRODUCT_TYPE'], [19, 23, 'I-PRODUCT_TYPE'], [24, 27, 'O'], [28, 32, 'B-SKIN_CONCERN'], [33, 37, 'O'], [38, 42, 'B-MIN_PRICE'], [42, 45, 'B-MAX_PRICE']]}


Entry 293:
Text: Looking for a cleanser for acne-prone skin $23-$52
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 22, 'B-PRODUCT_TYPE'], [23, 26, 'O'], [27, 37, 'B-SKIN_CONCERN'], [38, 42, 'O'], [43, 47, 'B-MIN_PRICE'], [47, 50, 'B-MAX_PRICE']]}


Entry 294:
Text: Looking for a face mask for combination skin $49-$67
Annotations: {'entities': [[0, 7, 'O'], [8, 11, 'O'], [12, 13, 'O'], [14, 18, 'B-PRODUCT_TYPE'], [19, 23, 'I-PRODUCT_TYPE'], [24, 27, 'O'], [28, 39, 'B-SKIN_CONCERN'], [40, 44, 'O'], [45, 49, 'B-MIN_PRICE'], [49, 52, 'B-MAX_PRICE']]}


Entry 295:
Text: Looking for a face mask for sensitive skin $37-$77 with Ceramides without Phthalates
Annotations: {'en

In [ ]:
#Checking structure
print(ner_classes)

['B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'B-SKIN_CONCERN', 'B-PREF_INGREDIENT', 'I-PREF_INGREDIENT', 'B-MIN_PRICE', 'B-MAX_PRICE', 'B-EXCL_INGREDIENT', 'I-EXCL_INGREDIENT', 'O']


In [ ]:
# Writing a function to split our annotations data into trainig, testing, validation
# split is roughly 70%/15%/15%

def split_annotations(data, train_size=210, test_size=45, val_size=45):
    """
    Splits the annotations data into training, testing, and validation sets.

    :param data: List of annotations.
    :param train_size: Size of the training set.
    :param test_size: Size of the test set.
    :param val_size: Size of the validation set.
    :return: Three lists representing the train, test, and validation sets.
    """
    total_length = len(data)
    all_indices = list(range(total_length))

    # Create evenly spaced indices for test and validation sets
    interval = total_length // (test_size + val_size)
    test_indices = all_indices[0:total_length:interval][:test_size]
    val_indices = all_indices[1:total_length:interval][:val_size]

    # Remaining indices for training set
    train_indices = list(set(all_indices) - set(test_indices) - set(val_indices))
    train_set = [data[i] for i in train_indices[:train_size]]

    # Create test and validation sets
    test_set = [data[i] for i in test_indices]
    val_set = [data[i] for i in val_indices]

    return train_set, test_set, val_set


In [ ]:
combo_df = annotated_df['annotations']
len(combo_df)

301

In [ ]:
# This actually splits 'annotations' data
train_df, test_df, val_df = split_annotations(annotated_df['annotations'])

# Check sizes of each df
len(train_df), len(test_df), len(val_df)

(210, 45, 45)

In [ ]:
train_df[:5]

[['Cleanser with seaweed more than $41 but less than $95\r',
  {'entities': [[0, 8, 'B-PRODUCT_TYPE'],
    [9, 13, 'O'],
    [14, 21, 'B-PREF_INGREDIENT'],
    [22, 26, 'O'],
    [27, 31, 'O'],
    [32, 35, 'B-MIN_PRICE'],
    [36, 39, 'O'],
    [40, 44, 'O'],
    [45, 49, 'O'],
    [50, 53, 'B-MAX_PRICE']]}],
 ['I need a moisturizer that includes Hyaluronic Acid for combination skin more than $29 but less than $48.\r',
  {'entities': [[0, 1, 'O'],
    [2, 6, 'O'],
    [7, 8, 'O'],
    [9, 20, 'B-PRODUCT_TYPE'],
    [21, 25, 'O'],
    [26, 34, 'O'],
    [35, 45, 'B-PREF_INGREDIENT'],
    [46, 50, 'I-PREF_INGREDIENT'],
    [51, 54, 'O'],
    [55, 66, 'B-SKIN_CONCERN'],
    [67, 71, 'O'],
    [72, 76, 'O'],
    [77, 81, 'O'],
    [82, 85, 'B-MIN_PRICE'],
    [86, 89, 'O'],
    [90, 94, 'O'],
    [95, 99, 'O'],
    [100, 103, 'B-MAX_PRICE'],
    [103, 104, 'O']]}],
 ["What's a good treatment to use for skin that's oily that includes Ceramides ?\r",
  {'entities': [[0, 4, 'O'],
    [4, 6, 

## Save dfs to Drive

Added in the two cells below to save this data in drive and be able to call it in other notebooks for testing/validation.

File folders must be created ahead of time within your google drive to do this and file names must match.  This isn't necessary to run.  I was experimenting with running stuff in a second notebook, but it ended up being more trouble than it was worth.


(Don't need to run second, do need first)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

# Function to save our split datasets retaining JSON filetype
def save_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file)

# Specify paths to save in Google Drive
train_path = '/content/drive/My Drive/DLAIEF23_Data/train_data.json'
test_path = '/content/drive/My Drive/DLAIEF23_Data/test_data.json'
val_path = '/content/drive/My Drive/DLAIEF23_Data/val_data.json'

# Call function & save
save_json(train_df, train_path)
save_json(test_df, test_path)
save_json(val_df, val_path)


# Model Main

Reminder to self of important names:

sephora_df

test_val_df

train_df

# Yet Another New Attempt at NER

In [ ]:
from transformers import AutoTokenizer, DistilBertForTokenClassification

# Initialize the tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize a blank model instance of DistilBertForTokenClassification
# The number of labels should match the number of unique labels in your NER task
num_labels = len(ner_classes)
model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=num_labels, return_dict=False)

# Define the global counter for None values (should end up discarding 3)
none_counter = 0

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Update the label_map to include 'I-' labels (not in our OG NER tags)
label_map = {
    'B-PRODUCT_TYPE': 0, 'I-PRODUCT_TYPE': 1,
    'B-SKIN_CONCERN': 2, 'I-SKIN_CONCERN': 3,  # Adding 'I-SKIN_CONCERN' here
    'B-PREF_INGREDIENT': 4, 'I-PREF_INGREDIENT': 5,
    'B-MIN_PRICE': 6, 'B-MAX_PRICE': 7,
    'B-EXCL_INGREDIENT': 8, 'I-EXCL_INGREDIENT': 9,
    'O': 10
}

In [ ]:
tokenizer("Hello, testing this sentence!")

{'input_ids': [101, 7592, 1010, 5604, 2023, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = []
    labels = []

    for example in examples:
        if example is None:
          continue  # Skip None items


        text = example[0]  # The text is the first element in the inner list
        entities = example[1]['entities']  # Entities are in a dictionary under the 'entities' key

        # Tokenize the text
        tokenized_input = tokenizer(text, truncation=True, is_split_into_words=False)
        tokenized_inputs.append(tokenized_input)

        # Initialize a list of -100 for each token
        label_ids = [-100] * len(tokenized_input['input_ids'])

        for start, end, label in entities:
            # Convert entity labels to integer IDs
            label_id = label_map[label]  # map labels like 'B-PRODUCT_TYPE' to integers

            # Align labels with tokens
            entity_tokens = tokenizer.encode(text[start:end], add_special_tokens=False)
            first_token = True
            for token in entity_tokens:
                try:
                    token_index = tokenized_input['input_ids'].index(token)
                    # Only label the first token of a given word
                    if first_token:
                        label_ids[token_index] = label_id
                        first_token = False
                    else:
                        # Optionally, label subsequent tokens as inside the entity
                        # label_ids[token_index] = label_map["I-" + label.split("-")[1]]
                        pass
                except ValueError:
                    continue

        labels.append(label_ids)

    # Combine the tokenized inputs into a single dictionary
    combined_tokenized_inputs = {key: [dic[key] for dic in tokenized_inputs] for key in tokenized_inputs[0]}
    combined_tokenized_inputs["labels"] = labels

    return combined_tokenized_inputs

In [ ]:
tokenized_data = tokenize_and_align_labels(train_df)
print(tokenized_data)

{'input_ids': [[101, 4550, 8043, 2007, 2712, 18041, 2062, 2084, 1002, 4601, 2021, 2625, 2084, 1002, 5345, 102], [101, 1045, 2342, 1037, 11052, 9496, 6290, 2008, 2950, 1044, 21095, 21017, 2594, 5648, 2005, 5257, 3096, 2062, 2084, 1002, 2756, 2021, 2625, 2084, 1002, 4466, 1012, 102], [101, 2054, 1005, 1055, 1037, 2204, 3949, 2000, 2224, 2005, 3096, 2008, 1005, 1055, 3514, 2100, 2008, 2950, 8292, 6444, 8621, 1029, 102], [101, 2054, 1005, 1055, 1037, 2204, 3949, 2000, 2224, 2005, 3096, 2008, 1005, 1055, 5257, 2008, 2987, 1005, 1056, 2031, 6544, 1029, 102], [101, 2559, 2005, 1037, 2227, 7308, 2005, 3514, 2100, 3096, 1002, 2382, 1011, 1002, 2871, 2302, 24980, 102], [101, 2559, 2005, 1037, 2227, 7308, 2005, 9353, 2638, 1011, 13047, 3096, 1002, 4464, 1011, 1002, 4466, 102], [101, 2559, 2005, 1037, 3239, 6949, 2005, 7591, 3096, 1002, 2570, 1011, 1002, 4720, 2007, 1044, 21095, 21017, 2594, 5648, 2302, 13012, 20464, 8820, 2078, 102], [101, 2559, 2005, 1037, 2227, 7308, 2005, 3514, 2100, 3096, 100

## NER New

In [ ]:
import re

# Deals with the $ in our min and max price NER tags
dollar_pattern = re.compile(r'\$[0-9]+')

# Custom tokenizer
def custom_tokenize(text, tokenizer):
    # Find all dollar amounts in the text
    dollar_matches = dollar_pattern.findall(text)

    # Split the text into tokens and dollar amounts
    tokens_and_dollars = re.split(r'(\$[0-9]+)', text)

    # Tokenize the text using the tokenizer
    tokens = []
    for item in tokens_and_dollars:
        if item.startswith('$'):
            # Replace dollar amount placeholders with the actual text
            tokens.append(item)
        else:
            tokens.extend(tokenizer.tokenize(item))

    return tokens

In [ ]:
# New function to handle the subtokenization from DistilBERT
# Our NER tags were done for whole words (ie: retinal had one tag)
# DistilBERT is breaking down our whole words into subtokens beginning with ##
# Then it's adding a B- and I- to the front of these subtokens
# So our new tokens end up looking like "B-B-" "I-B-"etc. for "R" "##eti" "##nol" (this might not be exact, but it's the idea)

def align_tokens_and_labels(tokenizer, sentence, labels):

    print("Original Sentence:", sentence)
    print("Original Labels:", labels)

    tokens = custom_tokenize(sentence, tokenizer)
    new_labels = []
    word_index = 0

    for token in tokens:
        print("Current Token:", token)

        if token.startswith("##"):   # flag the subtokens that are part of a word
            if word_index - 1 < len(labels) and labels[word_index - 1].startswith("B-"):
                # Change label from 'B-' to 'I-' for subtokens
                new_labels.append("I-" + labels[word_index - 1][2:])  #if our second label starts with B- change it to I- and drop the first
            elif word_index - 1 < len(labels):
                # Retain the label for subtokens
                new_labels.append(labels[word_index - 1])
            else:
                new_labels.append('O')  # Default label for out-of-range subtokens

            print("Subtoken, Label Assigned:", new_labels[-1])

        else:
            if word_index < len(labels):
                new_labels.append(labels[word_index])
            else:
                new_labels.append('O')  # Default label for out-of-range tokens
            word_index += 1

            print("Whole Token, Label Assigned:", new_labels[-1])

    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
    aligned_labels = ['O'] + new_labels + ['O']

    print("Processed Tokens:", tokens)
    print("Aligned Labels:", aligned_labels)

    return tokens, aligned_labels


In [ ]:
# debugging

# Sample
sample_sentence = "Cleanser with seaweed more than $41 but less than $95"
sample_labels = ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']

# Run the debugged function
tokens, aligned_labels = align_tokens_and_labels(tokenizer, sample_sentence, sample_labels)


Original Sentence: Cleanser with seaweed more than $41 but less than $95
Original Labels: ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']
Current Token: clean
Whole Token, Label Assigned: B-PRODUCT_TYPE
Current Token: ##ser
Subtoken, Label Assigned: I-PRODUCT_TYPE
Current Token: with
Whole Token, Label Assigned: O
Current Token: sea
Whole Token, Label Assigned: B-PREF_INGREDIENT
Current Token: ##weed
Subtoken, Label Assigned: I-PREF_INGREDIENT
Current Token: more
Whole Token, Label Assigned: O
Current Token: than
Whole Token, Label Assigned: O
Current Token: $41
Whole Token, Label Assigned: B-MIN_PRICE
Current Token: but
Whole Token, Label Assigned: O
Current Token: less
Whole Token, Label Assigned: O
Current Token: than
Whole Token, Label Assigned: O
Current Token: $95
Whole Token, Label Assigned: B-MAX_PRICE
Processed Tokens: ['[CLS]', 'clean', '##ser', 'with', 'sea', '##weed', 'more', 'than', '$41', 'but', 'less', 'than', '$95', '[

In [ ]:
# Debuggin

for label in aligned_labels:
    print(f"Label: {label}, Mapped to: {label_map.get(label, 'Unknown')}")


Label: O, Mapped to: 9
Label: B-PRODUCT_TYPE, Mapped to: 0
Label: I-PRODUCT_TYPE, Mapped to: 1
Label: O, Mapped to: 9
Label: B-PREF_INGREDIENT, Mapped to: 3
Label: I-PREF_INGREDIENT, Mapped to: 4
Label: O, Mapped to: 9
Label: O, Mapped to: 9
Label: B-MIN_PRICE, Mapped to: 5
Label: O, Mapped to: 9
Label: O, Mapped to: 9
Label: O, Mapped to: 9
Label: B-MAX_PRICE, Mapped to: 6
Label: O, Mapped to: 9


In [ ]:
# Test the function with the updated tokenizer
test_sentence = "Moisturizer with Hyaluronic Acid under $60 for dry skin"
test_labels =  ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'I-PREF_INGREDIENT', 'O', 'B-MAX_PRICE', 'O', 'B-SKIN_CONCERN', 'O']

tokens, aligned_labels = align_tokens_and_labels(tokenizer, test_sentence, test_labels)

# Make sure it worked with a print statement
for token, label in zip(tokens, aligned_labels):
    print(f"Token: {token}, Label: {label}")

Original Sentence: Moisturizer with Hyaluronic Acid under $60 for dry skin
Original Labels: ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'I-PREF_INGREDIENT', 'O', 'B-MAX_PRICE', 'O', 'B-SKIN_CONCERN', 'O']
Current Token: moist
Whole Token, Label Assigned: B-PRODUCT_TYPE
Current Token: ##uri
Subtoken, Label Assigned: I-PRODUCT_TYPE
Current Token: ##zer
Subtoken, Label Assigned: I-PRODUCT_TYPE
Current Token: with
Whole Token, Label Assigned: O
Current Token: h
Whole Token, Label Assigned: B-PREF_INGREDIENT
Current Token: ##yal
Subtoken, Label Assigned: I-PREF_INGREDIENT
Current Token: ##uron
Subtoken, Label Assigned: I-PREF_INGREDIENT
Current Token: ##ic
Subtoken, Label Assigned: I-PREF_INGREDIENT
Current Token: acid
Whole Token, Label Assigned: I-PREF_INGREDIENT
Current Token: under
Whole Token, Label Assigned: O
Current Token: $60
Whole Token, Label Assigned: B-MAX_PRICE
Current Token: for
Whole Token, Label Assigned: O
Current Token: dry
Whole Token, Label Assigned: B-SKIN_CONCERN
Cur

In [ ]:
# Also including a function to align the labels with padding
# Seems like we can benefit from padding in the model, but was having issues getting the NER to line up when including this
# This funciton is meant to rectify

def align_labels_with_padding(token_ids, labels, pad_token_id):
    """
    Aligns labels with token_ids, ensuring labels for padding tokens.
    :param token_ids: List of token ids.
    :param labels: List of labels aligned with original tokens.
    :param pad_token_id: The id of the padding token.
    :return: List of labels aligned with token_ids.
    """
    new_labels = []
    label_index = 0

    for token_id in token_ids:
        if token_id == tokenizer.cls_token_id or token_id == tokenizer.sep_token_id:
            new_labels.append(label_map['O'])
        elif token_id == pad_token_id:
            new_labels.append(label_map['O'])
        else:
            new_labels.append(label_map.get(labels[label_index], label_map['O']))
            label_index += 1

        # Debugging: Print token ID and aligned label
        print("Token ID:", token_id, "Aligned Label:", new_labels[-1])

    print("Aligned and Padded Labels:", new_labels)
    return new_labels


In [ ]:
# debugging

# Sample
sample_sentence = "Cleanser with seaweed more than $41 but less than $95"
sample_labels = ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']

# Run the debugged function
tokens, aligned_labels = align_tokens_and_labels(tokenizer, sample_sentence, sample_labels)

Original Sentence: Cleanser with seaweed more than $41 but less than $95
Original Labels: ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']
Current Token: clean
Whole Token, Label Assigned: B-PRODUCT_TYPE
Current Token: ##ser
Subtoken, Label Assigned: I-PRODUCT_TYPE
Current Token: with
Whole Token, Label Assigned: O
Current Token: sea
Whole Token, Label Assigned: B-PREF_INGREDIENT
Current Token: ##weed
Subtoken, Label Assigned: I-PREF_INGREDIENT
Current Token: more
Whole Token, Label Assigned: O
Current Token: than
Whole Token, Label Assigned: O
Current Token: $41
Whole Token, Label Assigned: B-MIN_PRICE
Current Token: but
Whole Token, Label Assigned: O
Current Token: less
Whole Token, Label Assigned: O
Current Token: than
Whole Token, Label Assigned: O
Current Token: $95
Whole Token, Label Assigned: B-MAX_PRICE
Processed Tokens: ['[CLS]', 'clean', '##ser', 'with', 'sea', '##weed', 'more', 'than', '$41', 'but', 'less', 'than', '$95', '[

In [ ]:
# Assuming tokenizer and label_map are already defined

# Sample data
sample_sentence = "Cleanser with seaweed more than $41 but less than $95"
sample_labels = ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']

# Run the align_tokens_and_labels function
tokens, aligned_labels = align_tokens_and_labels(tokenizer, sample_sentence, sample_labels)

print("Processed Tokens:", tokens)
print("Aligned Labels:", aligned_labels)


Original Sentence: Cleanser with seaweed more than $41 but less than $95
Original Labels: ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']
Current Token: clean
Whole Token, Label Assigned: B-PRODUCT_TYPE
Current Token: ##ser
Subtoken, Label Assigned: I-PRODUCT_TYPE
Current Token: with
Whole Token, Label Assigned: O
Current Token: sea
Whole Token, Label Assigned: B-PREF_INGREDIENT
Current Token: ##weed
Subtoken, Label Assigned: I-PREF_INGREDIENT
Current Token: more
Whole Token, Label Assigned: O
Current Token: than
Whole Token, Label Assigned: O
Current Token: $41
Whole Token, Label Assigned: B-MIN_PRICE
Current Token: but
Whole Token, Label Assigned: O
Current Token: less
Whole Token, Label Assigned: O
Current Token: than
Whole Token, Label Assigned: O
Current Token: $95
Whole Token, Label Assigned: B-MAX_PRICE
Processed Tokens: ['[CLS]', 'clean', '##ser', 'with', 'sea', '##weed', 'more', 'than', '$41', 'but', 'less', 'than', '$95', '[

In [ ]:
# Convert tokens to token IDs
sample_token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Run align_labels_with_padding
padded_labels = align_labels_with_padding(sample_token_ids, aligned_labels, tokenizer.pad_token_id)

# Debugging: Print the results
print("Token IDs:", sample_token_ids)
print("Padded Aligned Labels:", padded_labels)


Token ID: 101 Aligned Label: 9
Token ID: 4550 Aligned Label: 9
Token ID: 8043 Aligned Label: 0
Token ID: 2007 Aligned Label: 1
Token ID: 2712 Aligned Label: 9
Token ID: 18041 Aligned Label: 3
Token ID: 2062 Aligned Label: 4
Token ID: 2084 Aligned Label: 9
Token ID: 100 Aligned Label: 9
Token ID: 2021 Aligned Label: 5
Token ID: 2625 Aligned Label: 9
Token ID: 2084 Aligned Label: 9
Token ID: 100 Aligned Label: 9
Token ID: 102 Aligned Label: 9
Aligned and Padded Labels: [9, 9, 0, 1, 9, 3, 4, 9, 9, 5, 9, 9, 9, 9]
Token IDs: [101, 4550, 8043, 2007, 2712, 18041, 2062, 2084, 100, 2021, 2625, 2084, 100, 102]
Padded Aligned Labels: [9, 9, 0, 1, 9, 3, 4, 9, 9, 5, 9, 9, 9, 9]


In [ ]:
# function to call other functions to actually process the data
# includes various print statements from where i've done some troubleshooting

def process_ner_data(data, tokenizer, label_map, max_length=64):
    processed_data = []

    for item in data:
        if item is None:
            continue  # Skip None items

        sentence, entity_info = item
        entities = entity_info['entities']


        # Debugging: Check if entity indices are within the range
        for start, end, label in entities:
            if start >= len(sentence.split()) or end > len(sentence.split()):
                print(f"Entity indices out of range in sentence: '{sentence}'")
                print(f"Entity: Start: {start}, End: {end}, Label: {label}")

        # Convert indices in 'entities' to labels
        labels = ['O'] * len(sentence.split())  # Splitting the sentence into words
        for start, end, label in entities:
            if end <= len(labels):  # Ensure end index is within range
                labels[start] = label
                for i in range(start + 1, end):
                    labels[i] = 'I-' + label[2:] if label.startswith('B-') else label

        # Tokenize the sentence and align labels
        tokens, aligned_labels = align_tokens_and_labels(tokenizer, sentence, labels)

        # Tokenize with DistilBERT tokenizer
        tokenized_input = tokenizer(
            tokens,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            is_split_into_words=False,
            return_tensors='pt'
        )

        # Debugging: Check if tokenized input exceeds max_length
        if len(tokenized_input['input_ids'].squeeze()) > max_length:
            print(f"Warning: Tokenized input exceeds max_length for sentence: '{sentence}'")

        token_ids = tokenized_input['input_ids'].squeeze()  # Remove batch dimension
        attention_mask = tokenized_input['attention_mask'].squeeze()  # Remove batch dimension
        aligned_labels_padded = align_labels_with_padding(token_ids.tolist(), aligned_labels, tokenizer.pad_token_id)
        labels_tensor = torch.tensor([label_map.get(label, label_map['O']) for label in aligned_labels_padded])

        # Debugging: Check size and print first few labels
        if len(token_ids) != len(labels_tensor):
            print(f"Size mismatch in processed data for sentence: '{sentence}'")
        print("First few labels:", labels_tensor[:10])


        # Check for size mismatch
        if len(token_ids) != len(attention_mask) or len(token_ids) != len(labels_tensor):
            print(f"Found an issue with processed data at index {len(processed_data)}: Size mismatch.")
            continue

        processed_data.append((token_ids, attention_mask, labels_tensor))

        # Debugging: Inspect the first few processed data items
        for i, (token_ids, attention_mask, labels_tensor) in enumerate(processed_data[:5]):
            print(f"Sample {i}:")
            print("Token IDs shape:", token_ids.shape)
            print("Attention Mask shape:", attention_mask.shape)
            print("Labels Tensor shape:", labels_tensor.shape)

    return processed_data

# Process the data
processed_ner_data = process_ner_data(train_df, tokenizer, label_map)

# Check for issues in processed data
for item in processed_ner_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)


Streaming output truncated to the last 5000 lines.
Current Token: re
Whole Token, Label Assigned: O
Current Token: ##tino
Subtoken, Label Assigned: O
Current Token: ##l
Subtoken, Label Assigned: O
Current Token: for
Whole Token, Label Assigned: O
Current Token: ac
Whole Token, Label Assigned: O
Current Token: ##ne
Subtoken, Label Assigned: O
Current Token: -
Whole Token, Label Assigned: B-PRODUCT_TYPE
Current Token: prone
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: skin
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: more
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: than
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: $20
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: but
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: less
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: than
Whole Token, Label Assigned: O
Current Token: $66
Whole Token, Label Assigned: O
Current Token: .
Whole Token, Label Assigned: O

In [ ]:
# Debuggin full pipeline through process_ner
subset_processed_data = process_ner_data(train_df[:5], tokenizer, label_map)

# Debugging: Inspect processed data
for i, (token_ids, attention_mask, labels_tensor) in enumerate(subset_processed_data):
    print(f"Sample {i}:")
    print("Token IDs shape:", token_ids.shape)
    print("Attention Mask shape:", attention_mask.shape)
    print("Labels Tensor shape:", labels_tensor.shape)
    print("First few token IDs:", token_ids[:10])
    print("First few labels:", labels_tensor[:10])

# Continue with the rest of your analysis...


'
Entity: Start: 9, End: 13, Label: O
'
Entity: Start: 14, End: 21, Label: B-PREF_INGREDIENT
'
Entity: Start: 22, End: 26, Label: O
'
Entity: Start: 27, End: 31, Label: O
'
Entity: Start: 32, End: 35, Label: B-MIN_PRICE
'
Entity: Start: 36, End: 39, Label: O
'
Entity: Start: 40, End: 44, Label: O
'
Entity: Start: 45, End: 49, Label: O
'
Entity: Start: 50, End: 53, Label: B-MAX_PRICE
Original Sentence: Cleanser with seaweed more than $41 but less than $95
Original Labels: ['B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'O', 'O']
Current Token: clean
Whole Token, Label Assigned: B-PRODUCT_TYPE
Current Token: ##ser
Subtoken, Label Assigned: I-PRODUCT_TYPE
Current Token: with
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: sea
Whole Token, Label Assigned: I-PRODUCT_TYPE
Current Token: ##weed
Subtoken, Label Assigned: I-PRODUCT_TYPE
Current Token: more
Whole Token, Label Assigned: I-P

In [ ]:
# Check a few samples for consistency
for i, (token_ids, attention_mask, labels) in enumerate(processed_ner_data[:10]):
    print(f"Sample {i}:")
    print(f"Token IDs Length: {token_ids.size(0)}")
    print(f"Attention Mask Length: {attention_mask.size(0)}")
    print(f"Labels Length: {labels.size(0)}\n")

# Attempt to concatenate tensors
try:
    input_ids = torch.cat([x[0] for x in processed_ner_data], dim=0)
    attention_masks = torch.cat([x[1] for x in processed_ner_data], dim=0)
    labels = torch.cat([x[2] for x in processed_ner_data], dim=0)
    print("Concatenation successful. Data is consistent.")
except RuntimeError as e:
    print(f"Concatenation failed: {e}")


Getting to the lengths to match was a struggle, so I'm hesistant to change anything here... Unless absolutely necessary.

In [ ]:
# Create tensors for the data
input_ids = torch.cat([x[0] for x in processed_ner_data], dim=0)
attention_masks = torch.cat([x[1] for x in processed_ner_data], dim=0)
labels = torch.cat([x[2] for x in processed_ner_data], dim=0)

# debugging:
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Masks shape: {attention_masks.shape}")
print(f"Labels shape: {labels.shape}")


# Create the DataLoader
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=32)

# debugging

for batch in dataloader:
    b_input_ids, b_input_mask, b_labels = batch
    print(f"Batch Input IDs shape: {b_input_ids.shape}")
    print(f"Batch Attention Mask shape: {b_input_mask.shape}")
    print(f"Batch Labels shape: {b_labels.shape}")
    break  # Just to check the first batch


# Ready for model training!!!

In [ ]:
# debugging
samples = train_df[:5]  # Adjust as necessary
print(samples)

In [ ]:
# debugging

for i, item in enumerate(samples):
    print(f"--- Processing Sample {i+1} ---")

    # Process each item
    sentence, entity_info = item
    entities = entity_info['entities']

    # Convert indices in 'entities' to labels
    labels = ['O'] * len(sentence.split())
    # Continue with your label processing...

    # Tokenize and align labels
    tokens, aligned_labels = align_tokens_and_labels(tokenizer, sentence, labels)
    print("Tokens:", tokens)
    print("Aligned Labels before padding:", aligned_labels)

    # Tokenize with DistilBERT tokenizer and align labels with padding
    tokenized_input = tokenizer(tokens, add_special_tokens=True, max_length=64, padding='max_length', truncation=True, return_tensors='pt')
    token_ids = tokenized_input['input_ids'].squeeze()
    attention_mask = tokenized_input['attention_mask'].squeeze()
    aligned_labels_padded = align_labels_with_padding(token_ids.tolist(), aligned_labels, tokenizer.pad_token_id)
    print("Token IDs:", token_ids)
    print("Aligned Labels after padding:", aligned_labels_padded)

    # Check length consistency
    print(f"Length Check - Tokens: {len(token_ids)}, Labels: {len(aligned_labels_padded)}\n")


In [ ]:

sample_data = train_df[0]  # Taking the first sample for testing
sentence, entity_info = sample_data
entities = entity_info['entities']

# Convert indices in 'entities' to labels
labels = ['O'] * len(sentence.split())
for start, end, label in entities:
    if end <= len(labels):
        labels[start] = label
        for i in range(start + 1, end):
            labels[i] = 'I-' + label[2:] if label.startswith('B-') else label

# Tokenize the sentence and align labels
tokens, aligned_labels = align_tokens_and_labels(tokenizer, sentence, labels)

# Convert tokens to token IDs for the padding alignment function
sample_token_ids = tokenizer.convert_tokens_to_ids(tokens)


## Training with BERT

Here's where the fun stuff happens

In [ ]:
from transformers import DistilBertForTokenClassification, AdamW

# Define the number of epochs
epochs = 100  # adjust as needed later

# Define the model for DistilBERT
model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_map),  # Number of distinct labels in dataset
    output_attentions=False,
    output_hidden_states=False
)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set training params
optimizer = AdamW(model.parameters(), lr=1e-5)  # can adjust the learning rate as we train...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#set to use GPU when we can
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

Set-up WandB before training

In [ ]:
# Initialize a new run
run = wandb.init(project="DLAIE Project F23")

# Track hyperparameters and run metadata
wandb.config = {
  "learning_rate": 1e-5,
  "epochs": 100,
  "batch_size": 32
}

wandb: Currently logged in as: vcrawfordnelson (dlaief23). Use `wandb login --relogin` to force relogin


In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        print(f"Original Labels shape: {b_labels.shape}")


        # Reshape labels if necessary
        if b_labels.dim() == 1:
            b_labels = b_labels.view(-1, 64)  # Adjust 64 to sequence length if different

        # Print shapes for debugging
        print(f"Input IDs shape: {b_input_ids.shape}")
        print(f"Attention Mask shape: {b_input_mask.shape}")
        print(f"Labels shape: {b_labels.shape}")

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Log batch loss to W&B
        wandb.log({"batch_loss": loss.item()})

        # Backward pass and update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Calculate the average loss over the epoch
    avg_train_loss = total_loss / len(dataloader)

    # Log average training loss to W&B
    wandb.log({"epoch": epoch, "avg_train_loss": avg_train_loss})

    print(f"Average training loss for epoch {epoch}: {avg_train_loss}")


Original Labels shape: torch.Size([32])


RuntimeError: ignored

In [ ]:
# OG training loop
#model is learning from our data
#iterate over the DataLoader, passing data to model
#update model's weights based on the output

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Log batch loss to W&B
        wandb.log({"batch_loss": loss.item()})

        # Backward pass and update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Calculate the average loss over the epoch
    avg_train_loss = total_loss / len(dataloader)

    # Log average training loss to W&B
    wandb.log({"epoch": epoch, "avg_train_loss": avg_train_loss})

    print(f"Average training loss for epoch {epoch}: {avg_train_loss}")

ValueError: ignored

In [ ]:
# this cell saves the model we just trained to a google drive folder
# folder must be set up and named acordingly ahead of time!

# this allows us to call that specific model later and use it for validation, evaluation, etc.

# setting this equal to wandb's name for the run
run_name = run.name

from google.colab import drive
drive.mount('/content/drive')

# Create a directory name with the W&B run name in Google Drive
model_save_path = f'/content/drive/My Drive/DLAIEF23_Models/{run_name}'

# Save the model to this directory
model.save_pretrained(model_save_path)

In [ ]:
# Finish the W&B run
wandb.finish()

# Validation
val_df

NER section MUST run before for this to work.

Can skip training section as written.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForTokenClassification

# calling our saved model
model_path = '/content/drive/MyDrive/DLAIEF23_Models/dulcet-aardvark-32'

# Initialize the tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# Import our model
model = DistilBertForTokenClassification.from_pretrained(model_path)


In [ ]:
# call process_annotation function from earlier for validation data
processed_validation_data = [process_annotation(annotation) for annotation in val_df if annotation is not None]

# Print the number of None values discarded
print(f"Number of None values discarded: {none_counter}")

Number of None values discarded: 0


In [ ]:
#double check we've dropped our none's
for item in processed_validation_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)


In [ ]:
# Extract labels from processed val data
extracted_val_labels = [aligned_labels for _, _, aligned_labels in processed_validation_data if aligned_labels is not None]
print(extracted_val_labels)

[['B-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'I-B-SKIN_CONCERN', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'I-B-PRODUCT_TYPE', 'B-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'I-I-PRODUCT_TYPE', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MIN_PRICE', 'I-B-MIN_PRICE', 'I-B-MIN_PRICE', 'B-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'B-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'I-B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
# Correct the labels
corrected_val_labels = correct_labels(extracted_val_labels)
print(corrected_val_labels)

[['B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'B-SKIN_CONCERN', 'O', 'O', 'O', 'O', 'O', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'I-PRODUCT_TYPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MIN_PRICE', 'B-MIN_PRICE', 'B-MIN_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'B-MAX_PRICE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [ ]:
# this uses same label_map as training to ensure consistency
# Convert the corrected labels to numeric indices
numeric_val_labels = convert_labels_to_indices(corrected_val_labels, label_map)

In [ ]:
# Convert validation data to tensors
val_input_ids, val_attention_mask, _ = zip(*processed_validation_data)
val_input_ids_tensor = torch.tensor(val_input_ids, dtype=torch.long)
val_attention_mask_tensor = torch.tensor(val_attention_mask, dtype=torch.long)
val_labels_tensor = torch.tensor(numeric_val_labels, dtype=torch.long)

# Create a DataLoader for the validation data
validation_dataset = TensorDataset(val_input_ids_tensor, val_attention_mask_tensor, val_labels_tensor)
validation_dataloader = DataLoader(validation_dataset, batch_size=32)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch

# Set the model to use the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Put model in evaluation mode
model.eval()

# Disable gradient calculations
with torch.no_grad():
    # Initialize lists to store true labels and predictions
    true_labels = []
    predictions = []

    # Iterate over batches in the validation dataloader
    for batch in validation_dataloader:  # Use validation_dataloader here
        # Unpack the batch data and move it to the GPU
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass, get model predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Convert logits to actual predictions
        batch_predictions = np.argmax(logits, axis=2)

        # Append batch predictions and true labels
        true_labels.append(label_ids)
        predictions.append(batch_predictions)


In [ ]:
# Flatten the lists of true labels and predictions
true_labels_flat = [label for batch in true_labels for label in batch.flatten()]
predictions_flat = [prediction for batch in predictions for prediction in batch.flatten()]

In [ ]:
# Filtering out 'ignore_index' (-100 for DistilBERT which is HuggingFace)
true_labels_filtered = [label for label in true_labels_flat if label != -100]
predictions_filtered = [pred for pred, label in zip(predictions_flat, true_labels_flat) if label != -100]


In [ ]:
# Calculate metrics
accuracy = accuracy_score(true_labels_filtered, predictions_filtered)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels_filtered, predictions_filtered, average='weighted')


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f"Validation Accuracy: {accuracy}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1-Score: {f1}")

Validation Accuracy: 0.9844618055555555
Validation Precision: 0.985064601644672
Validation Recall: 0.9844618055555555
Validation F1-Score: 0.9845083532090857


# Evaluation

NER section MUST run before for this to work.

Can skip training & validation sections as written.

## Set Up

Call our favorite model, make sure it's in evaluation mode

In [ ]:
from transformers import DistilBertForTokenClassification, DistilBertTokenizer

# Calling our model here so we can run this without running traning and validation cells
model_path = '/content/drive/MyDrive/DLAIEF23_Models/dulcet-aardvark-32'

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForTokenClassification.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [ ]:
# Example user input - using this to test the functions I'm writing
user_input = "I'm looking for an eye cream with retinol"


In [ ]:
# Invert the label_map to map indices to labels
index_to_label_map = {index: label for label, index in label_map.items()}


In [ ]:
# Tokenize and run the model
inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=512)
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)

# Convert predictions to labels
predicted_indices = predictions[0].tolist()
predicted_labels = [index_to_label_map[idx] for idx in predicted_indices]

# Tokenize the input text to match with predictions
tokenized_input = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [ ]:
entities = {}
current_entity = ""
current_type = None
for token, label in zip(tokenized_input, predicted_labels):
    if label.startswith("B-"):
        if current_entity and current_type:
            entities.setdefault(current_type, []).append(current_entity)
        current_entity = token.lstrip("##")  # Start a new entity
        current_type = label.split("-")[1]
    elif label.startswith("I-") and current_type == label.split("-")[1]:
        current_entity += " " + token.lstrip("##")  # Append to the current entity
    else:
        if current_entity and current_type:
            entities.setdefault(current_type, []).append(current_entity)
        current_entity = ""
        current_type = None

# Add the last entity if it exists
if current_entity and current_type:
    entities.setdefault(current_type, []).append(current_entity)

print("Extracted entities:", entities)
for token, label in zip(tokenized_input, predicted_labels):
    print(f"{token}: {label}")


Extracted entities: {'PRODUCT_TYPE': ['tino', 'l', '[SEP]']}
[CLS]: O
i: O
': O
m: O
looking: O
for: O
an: O
eye: O
cream: O
with: O
re: O
##tino: B-PRODUCT_TYPE
##l: B-PRODUCT_TYPE
[SEP]: B-PRODUCT_TYPE


## Product Type

In [ ]:
def filter_by_product_type(df, product_types):
    """
    Filter the DataFrame based on product type, handling multi-word labels.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    product_types (list): List of product types extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Normalize product types for case-insensitive comparison
        normalized_product_types = {ptype.replace(" ", "").lower() for ptype in product_types}

        # Function to check if the label matches any product type
        def label_matches(label):
            normalized_label = label.replace(" ", "").lower()
            return normalized_label in normalized_product_types

        # Filter DataFrame
        filtered_df = df[df['Label'].apply(label_matches)]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_product_type: {e}")
        return df  # Return the original DataFrame in case of error


## Preferred Ingredient

In [ ]:
def filter_by_pref_ingredient(df, pref_ingredients):
    """
    Filter the DataFrame based on preferred ingredients.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    pref_ingredients (list): List of preferred ingredients extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Normalize preferred ingredients for case-insensitive comparison
        normalized_pref_ingredients = {ingredient.lower() for ingredient in pref_ingredients}

        # Function to check if the ingredients list contains any preferred ingredient
        def contains_pref_ingredient(ingredient_list):
           # Normalize and split the ingredients list for comparison
           ingredients = set(ing.lower() for ing in ingredient_list.split(', '))
           return any(ing in ingredients for ing in normalized_pref_ingredients)

        # Filter DataFrame
        filtered_df = df[df['ingredients'].apply(contains_pref_ingredient)]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_pref_ingredient: {e}")
        return df  # Return the original DataFrame in case of error


## Excluded Ingredients

In [ ]:
def filter_by_excl_ingredient(df, excl_ingredients):
    """
    Filter the DataFrame to exclude products with certain ingredients.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    excl_ingredients (list): List of excluded ingredients extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Normalize excluded ingredients for case-insensitive comparison
        normalized_excl_ingredients = {ing.lower() for ing in excl_ingredients}

        # Function to check if the ingredients list contains any excluded ingredient
        def contains_excl_ingredient(ingredient_list):
            # Normalize and split the ingredients list for comparison
            ingredients = set(ing.lower() for ing in ingredient_list.split(', '))
            return not any(ing in ingredients for ing in normalized_excl_ingredients)

        # Filter DataFrame
        filtered_df = df[df['ingredients'].apply(contains_excl_ingredient)]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_excl_ingredient: {e}")
        return df  # Return the original DataFrame in case of error


## Min Price

In [ ]:
def filter_by_min_price(df, min_price):
    """
    Filter the DataFrame to include only products with a price greater than or equal to min_price.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    min_price (float): The minimum price threshold.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Convert price from string to float and compare
        filtered_df = df[df[' price '].replace('[\$,]', '', regex=True).astype(float) >= min_price]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_min_price: {e}")
        return df

## Max Price

In [ ]:
def filter_by_max_price(df, max_price):
    """
    Filter the DataFrame to include only products with a price less than or equal to max_price.

    Args:
    df (DataFrame): The DataFrame to filter (sephora_df).
    max_price (float): The maximum price threshold.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    try:
        # Convert price from string to float and compare
        filtered_df = df[df[' price '].replace('[\$,]', '', regex=True).astype(float) <= max_price]
        return filtered_df
    except Exception as e:
        print(f"Error in filter_by_max_price: {e}")
        return df

## Skin Concerns

In [ ]:
def filter_by_skin_concern(sephora_df, ingredient_mapping_df, skin_concerns):
    """
    Filter the DataFrame based on skin concerns.

    Args:
    sephora_df (DataFrame): The DataFrame of products (sephora_df).
    ingredient_mapping_df (DataFrame): The DataFrame mapping skin concerns to preferred and excluded ingredients.
    skin_concerns (list): List of skin concerns extracted from the NER model.

    Returns:
    DataFrame: Filtered DataFrame.
    """
    # Normalize skin concerns for case-insensitive comparison
    normalized_skin_concerns = {concern.lower() for concern in skin_concerns}

    # Filter ingredient_mapping_df for these concerns
    concern_filter = ingredient_mapping_df['Skin Concern'].str.lower().apply(lambda x: any(concern in x for concern in normalized_skin_concerns))
    filtered_ingredient_mapping = ingredient_mapping_df[concern_filter]

    # Aggregate preferred and excluded ingredients
    preferred_ingredients = set()
    excluded_ingredients = set()
    for _, row in filtered_ingredient_mapping.iterrows():
        preferred_ingredients.update(row['Preferred Ingredients'].split(', '))
        excluded_ingredients.update(row['Excluded Ingredients'].split(', '))

    # Apply filters on sephora_df
    if preferred_ingredients:
        sephora_df = filter_by_pref_ingredient(sephora_df, list(preferred_ingredients))
    if excluded_ingredients:
        sephora_df = filter_by_excl_ingredient(sephora_df, list(excluded_ingredients))

    return sephora_df


## Combining & Calling Filters

In [ ]:
# Filter by Product Type
if 'PRODUCT_TYPE' in entities:
    filtered_sephora_df = filter_by_product_type(sephora_df, entities['PRODUCT_TYPE'])

# Filter by Preferred Ingredient
if 'PREF_INGREDIENT' in entities:
    filtered_sephora_df = filter_by_pref_ingredient(filtered_sephora_df, entities['PREF_INGREDIENT'])

# Filter by Excluded Ingredient
excl_ingredients = []
if 'EXCL_INGREDIENT' in entities:
    for ing in entities['EXCL_INGREDIENT']:
        # Concatenate multi-word ingredients if they are split
        excl_ingredients.append(ing)
#filter the DataFrame
filtered_sephora_df = filter_by_excl_ingredient(filtered_sephora_df, excl_ingredients)

# Extraction of MIN_PRICE and MAX_PRICE
min_price = float(entities['MIN_PRICE'][0]) if 'MIN_PRICE' in entities else None
max_price = float(entities['MAX_PRICE'][0]) if 'MAX_PRICE' in entities else None

# Apply price filters
if min_price is not None:
    filtered_sephora_df = filter_by_min_price(filtered_sephora_df, min_price)

if max_price is not None:
    filtered_sephora_df = filter_by_max_price(filtered_sephora_df, max_price)

if 'SKIN_CONCERN' in entities:
    filtered_sephora_df = filter_by_skin_concern(filtered_sephora_df, ingredient_mapping_df, entities['SKIN_CONCERN'])

# Print the final filtered DataFrame
print("DataFrame after Skin Concern Filtering:", filtered_sephora_df)


DataFrame after Skin Concern Filtering: Empty DataFrame
Columns: []
Index: []


In [ ]:
# Manually set entity values for testing
test_entities = {
    'PRODUCT_TYPE': ['moisturizer'],
    'PREF_INGREDIENT': ['hyaluronic acid'],
    'EXCL_INGREDIENT': ['parabens'],
    'MIN_PRICE': ['20'],  # Representing as strings to simulate model output
    'MAX_PRICE': ['50'],
    'SKIN_CONCERN': ['dryness']
}

# Apply all filters based on test entities
filtered_sephora_df = sephora_df.copy()  # Start with the original DataFrame

if 'PRODUCT_TYPE' in test_entities:
    filtered_sephora_df = filter_by_product_type(filtered_sephora_df, test_entities['PRODUCT_TYPE'])

if 'PREF_INGREDIENT' in test_entities:
    filtered_sephora_df = filter_by_pref_ingredient(filtered_sephora_df, test_entities['PREF_INGREDIENT'])

if 'EXCL_INGREDIENT' in test_entities:
    excl_ingredients = test_entities['EXCL_INGREDIENT']
    filtered_sephora_df = filter_by_excl_ingredient(filtered_sephora_df, excl_ingredients)

if 'MIN_PRICE' in test_entities:
    min_price = float(test_entities['MIN_PRICE'][0])
    filtered_sephora_df = filter_by_min_price(filtered_sephora_df, min_price)

if 'MAX_PRICE' in test_entities:
    max_price = float(test_entities['MAX_PRICE'][0])
    filtered_sephora_df = filter_by_max_price(filtered_sephora_df, max_price)

if 'SKIN_CONCERN' in test_entities:
    filtered_sephora_df = filter_by_skin_concern(filtered_sephora_df, ingredient_mapping_df, test_entities['SKIN_CONCERN'])

# Print the final filtered DataFrame
print("Final Filtered DataFrame after all tests:", filtered_sephora_df)


Final Filtered DataFrame after all tests:             Label               brand  \
469   Moisturizer    FIRST AID BEAUTY   
597   Moisturizer        IT COSMETICS   
598   Moisturizer        IT COSMETICS   
888   Moisturizer        OLEHENRIKSEN   
1039  Moisturizer          PHILOSOPHY   
1087  Moisturizer  SEPHORA COLLECTION   
1244  Moisturizer               TARTE   

                                                   name    price   rank  \
469                       Ultra Repair® Hydrating Serum   $36.00    4.2   
597   Your Skin But Better™ CC+ Airbrush Perfecting ...   $35.00    3.6   
598      Bye Bye Redness™ Neutralizing Correcting Cream   $32.00    4.2   
888                       C-Rush™ Brightening Gel Crème   $44.00    4.6   
1039  Renewed Hope in A Jar Refreshing & Refining Mo...   $39.00    4.1   
1087              Intensive Instant Moisturizer + Cream   $23.00    4.3   
1244  Drink of H2O Hydrating Boost Moisturizer Rainf...   $39.00    4.4   

                            

In [ ]:
# Print the first few entries of the price column
print(sephora_df[' price '].head())


0     $112.00 
1      $94.00 
2      $28.00 
3      $95.00 
4      $98.00 
Name:  price , dtype: object


# Old Code

## NER - Old - Don't run!!!

### Old Code - Don't use

Goes with NER New but replaced by stuff that works better, saving in case need to bring this back during troubleshooting

In [ ]:
# Don't run!!!
# saving was working before in NER new

def align_tokens_and_labels(tokenizer, sentence, labels):
    tokens = custom_tokenize(sentence, tokenizer)
    new_labels = []
    word_index = 0

    for token in tokens:
        if token.startswith("##"):
            if labels[word_index - 1].startswith("B-"):
                # Change label from 'B-' to 'I-' for subtokens
                new_labels.append("I-" + labels[word_index - 1][2:])
            else:
                # Retain the label for subtokens
                new_labels.append(labels[word_index - 1])
        else:
            if word_index < len(labels):
                new_labels.append(labels[word_index])
            else:
                new_labels.append('O')  # Default label for extra tokens
            word_index += 1

    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
    aligned_labels = ['O'] + new_labels + ['O']

    return tokens, aligned_labels


In [ ]:
# saving because was working close before

def process_ner_data(data, tokenizer, label_map, max_length=64):
    processed_data = []

    for item in data:
        if item is None:
            continue  # Skip None items

        sentence, entity_info = item
        entities = entity_info['entities']

        # Convert indices in 'entities' to labels
        labels = ['O'] * len(sentence)
        for start, end, label in entities:
            labels[start] = label
            for i in range(start + 1, end):
                if label.startswith('B-'):
                    labels[i] = 'I-' + label[2:]
                else:
                    labels[i] = label

        # Tokenize the sentence and align labels
        tokens, aligned_labels = align_tokens_and_labels(tokenizer, sentence, labels)

        # Tokenize with DistilBERT tokenizer
        tokenized_input = tokenizer(
            tokens,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            is_split_into_words=False,
            return_tensors='pt'
        )

        token_ids = tokenized_input['input_ids'].view(1, -1)
        attention_mask = tokenized_input['attention_mask'].view(1, -1)
        aligned_labels_padded = align_labels_with_padding(token_ids[0].tolist(), aligned_labels, tokenizer.pad_token_id)
        labels_tensor = torch.tensor([label_map.get(label, label_map['O']) for label in aligned_labels_padded]).unsqueeze(0)

        processed_data.append((token_ids, attention_mask, labels_tensor))

    return processed_data


print(f"Total samples in train_df: {len(train_df)}")

# Process the data
processed_ner_data = process_ner_data(train_df, tokenizer, label_map)

# Check for issues in processed data
for item in processed_ner_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)


Total samples in train_df: 210


### Entire OG NER Section

Also no longer using, but keeping in case need to restore to this

In [ ]:
from transformers import AutoTokenizer, DistilBertForTokenClassification

# Initialize the tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize a blank model instance of DistilBertForTokenClassification
# The number of labels should match the number of unique labels in your NER task
num_labels = len(ner_classes)
model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=num_labels, return_dict=False)

# Define the global counter for None values (should end up discarding 3)
none_counter = 0


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Sample test sentences
test_sentences = [
    "Treatment with Retinol more than $43 but less than $92",
    "I need a moisturizer that's under $50 with hyaluronic acid",
    "Looking for a cleanser without sulfates costing between $20 and $35"
]

# Tokenizing each sentence and printing the tokens
for sentence in test_sentences:
    tokenized_output = tokenizer.tokenize(sentence)
    print(f"Sentence: {sentence}")
    print(f"Tokens: {tokenized_output}\n")


Sentence: Treatment with Retinol more than $43 but less than $92
Tokens: ['treatment', 'with', 're', '##tino', '##l', 'more', 'than', '$', '43', 'but', 'less', 'than', '$', '92']

Sentence: I need a moisturizer that's under $50 with hyaluronic acid
Tokens: ['i', 'need', 'a', 'moist', '##uri', '##zer', 'that', "'", 's', 'under', '$', '50', 'with', 'h', '##yal', '##uron', '##ic', 'acid']

Sentence: Looking for a cleanser without sulfates costing between $20 and $35
Tokens: ['looking', 'for', 'a', 'clean', '##ser', 'without', 'sulfate', '##s', 'costing', 'between', '$', '20', 'and', '$', '35']



In [ ]:
import re

# Define the dollar amount pattern
dollar_pattern = re.compile(r'\$[0-9]+')

# Custom tokenizer that preserves specific patterns (like dollar amounts)
def custom_tokenize(text, tokenizer):
    # Find all dollar amounts in the text
    dollar_matches = dollar_pattern.findall(text)

    # Split the text into tokens and dollar amounts
    tokens_and_dollars = re.split(r'(\$[0-9]+)', text)

    # Tokenize the text using the tokenizer
    tokens = []
    for item in tokens_and_dollars:
        if item.startswith('$'):
            # Replace dollar amount placeholders with the actual text
            tokens.append(item)
        else:
            tokens.extend(tokenizer.tokenize(item))

    return tokens

def align_tokens_and_labels(tokenizer, sentence, labels):
    # Use the custom tokenizer to tokenize the sentence
    tokens = custom_tokenize(sentence, tokenizer)

    new_labels = []
    word_index = 0

    for token in tokens:
        if token.startswith("##"):
            # Continuation of the previous word gets the same label
            new_labels.append(labels[word_index - 1])
        else:
            # New word gets the next label, if available
            if word_index < len(labels):
                new_labels.append(labels[word_index])
            else:
                new_labels.append('O')  # Default label for extra tokens
            word_index += 1

    # Add special tokens ([CLS] at the start and [SEP] at the end)
    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
    aligned_labels = ['O'] + new_labels + ['O']

    return tokens, aligned_labels

# Test the function with the updated tokenizer
test_sentence = "Treatment with Retinol more than $43 but less than $92"
test_labels = ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']

tokens, aligned_labels = align_tokens_and_labels(tokenizer, test_sentence, test_labels)

# Print the tokens and aligned labels for inspection
for token, label in zip(tokens, aligned_labels):
    print(f"Token: {token}, Label: {label}")


Token: [CLS], Label: O
Token: treatment, Label: B-PRODUCT_TYPE
Token: with, Label: O
Token: re, Label: B-PREF_INGREDIENT
Token: ##tino, Label: B-PREF_INGREDIENT
Token: ##l, Label: B-PREF_INGREDIENT
Token: more, Label: O
Token: than, Label: O
Token: $43, Label: B-MIN_PRICE
Token: but, Label: O
Token: less, Label: O
Token: than, Label: O
Token: $92, Label: B-MAX_PRICE
Token: [SEP], Label: O


In [ ]:
def align_labels_with_padding(token_ids, labels, pad_token_id):
    new_labels = []
    special_tokens_mask = [1 if token_id in [tokenizer.cls_token_id, tokenizer.sep_token_id] else 0 for token_id in token_ids]

    label_index = 0
    for mask in special_tokens_mask:
        if mask == 1:
            # For special tokens, use the 'O' label (or a label indicating 'no entity')
            new_labels.append('O')
        elif token_ids[label_index] == pad_token_id:
            # For padding tokens, use the 'O' label (or a label indicating 'no entity')
            new_labels.append('O')
        else:
            new_labels.append(labels[label_index])
            label_index += 1

    return new_labels


In [ ]:
def process_annotation(annotation, index, tokenizer):
    global none_counter

    # Check if the annotation is None or not formatted correctly
    if annotation is None or len(annotation) != 2:
        none_counter += 1
        return None

    # Extract sentence and entities from the annotation
    sentence, entity_info = annotation
    entities = entity_info['entities']

    # Ensure that 'sentence' is a string
    if isinstance(sentence, list):
        sentence = ' '.join(sentence)
    elif not isinstance(sentence, str):
        print(f"Error in annotation {index}: Sentence is neither a string nor a list.")
        return None

    # Check if sentence is a string before passing to custom_tokenize
    if not isinstance(sentence, str):
        raise ValueError(f"Expected a string for sentence, got {type(sentence)} instead.")

    # Use the custom tokenizer
    custom_tokenized_sentence = custom_tokenize(sentence, tokenizer)

    # Convert the list of tokens back to a string
    tokenized_sentence_str = " ".join(custom_tokenized_sentence)

    # Tokenize the sentence
    tokenized_input = tokenizer.encode_plus(
        tokenized_sentence_str,
        add_special_tokens=True,
        max_length=64,  # Adjust max_length as needed
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        is_split_into_words=False,
        return_tensors='pt'
    )

    # Initialize labels for each token as 'O'
    labels = ['O'] * len(tokenized_input["input_ids"].tolist()[0])

    # Iterate over the entities and update labels
    for start, end, label in entities:
        if start < len(labels):
            labels[start] = label  # Assign label directly without adding 'B-' or 'I-'

    # Align tokens with labels
    tokens, aligned_labels = align_tokens_and_labels(tokenizer, sentence.split(), labels)

    token_ids = tokenized_input['input_ids']
    attention_mask = tokenized_input['attention_mask']

    # Align labels with token_ids (taking care of special tokens and padding)
    aligned_labels_padded = align_labels_with_padding(token_ids[0].tolist(), aligned_labels, tokenizer.pad_token_id)

    return token_ids, attention_mask, aligned_labels_padded


for i, ann in enumerate(train_df):
    processed_data = process_annotation(ann, i, tokenizer)
    if processed_data:
        print(processed_data)


TypeError: ignored

In [ ]:
def print_sample_data(processed_data, sample_size=5):
    """
    Function to print a sample of processed data for manual inspection.
    :param processed_data: List of processed data tuples (token_ids, attention_masks, labels)
    :param sample_size: Number of samples to print
    """
    # Randomly select sample_size examples
    sample_indices = np.random.choice(len(processed_data), sample_size, replace=False)

    for index in sample_indices:
        token_ids, attention_mask, labels = processed_data[index]
        token_list = tokenizer.convert_ids_to_tokens(token_ids[0])

        print("\nSample:", index)
        print("{:<20} {:<15}".format("Token", "Label"))
        print("-"*35)

        for token, label in zip(token_list, labels):
            print("{:<20} {:<15}".format(token, label))



In [ ]:
# Process all annotations, excluding None values entirely
processed_data = [process_annotation(annotation) for annotation in train_df if annotation is not None]

# Print the number of None values discarded
print(f"Number of None values discarded: {none_counter}")

TypeError: ignored

In [ ]:
sentence = "Treatment with Retinol more than $43 but less than $92"
labels = ['B-PRODUCT_TYPE', 'O', 'B-PREF_INGREDIENT', 'O', 'O', 'B-MIN_PRICE', 'O', 'O', 'O', 'B-MAX_PRICE']

# Split the sentence into words
words = sentence.split()

# Call align_tokens_and_labels
tokens, aligned_labels = align_tokens_and_labels(tokenizer, words, labels)

# Print and check
for token, label in zip(tokens, aligned_labels):
    print(f"Token: {tokenizer.decode(token)}, Label: {label}")


TypeError: ignored

In [ ]:
#double check we've dropped our none's
for item in processed_data:
    if item is None or not isinstance(item, tuple) or len(item) != 3:
        print("Found an issue with processed data:", item)

In [ ]:
# Function to convert labels to indices
def convert_labels_to_indices(labels, label_map):
    return [[label_map[label] for label in label_seq] for label_seq in labels]

In [ ]:
# Extract labels from processed annotations
extracted_labels = [aligned_labels for _, _, aligned_labels in processed_data if aligned_labels is not None]
print(extracted_labels)

[['O', 'O', 'B-PRODUCT_TYPE', 'B-PRODUCT_TYPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [ ]:
# Create a mapping from string labels to unique integers
label_map = {label: i for i, label in enumerate(ner_classes)}
print(label_map)

{'B-PRODUCT_TYPE': 0, 'I-PRODUCT_TYPE': 1, 'B-SKIN_CONCERN': 2, 'B-PREF_INGREDIENT': 3, 'I-PREF_INGREDIENT': 4, 'B-MIN_PRICE': 5, 'B-MAX_PRICE': 6, 'B-EXCL_INGREDIENT': 7, 'I-EXCL_INGREDIENT': 8, 'O': 9}


In [ ]:
# Convert the corrected labels to numeric indices
numeric_labels = convert_labels_to_indices(extracted_labels, label_map)

In [ ]:
# Convert data to tensors and create a DataLoader
input_ids, attention_mask, _ = zip(*processed_data)
input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)
labels_tensor = torch.tensor(numeric_labels, dtype=torch.long)

dataset = TensorDataset(input_ids_tensor, attention_mask_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=32)

TypeError: ignored